In [1]:
import numpy as np
import pandas as pd
import torch
import tensorly as tl
# library for TTD   (Tensorly  (search))

from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tensorly.decomposition import tensor_train
from tensorly.decomposition import tensor_train
# different forms of tensor
from sklearn.preprocessing import MinMaxScaler
from tensorly import tt_to_tensor
from tensorly.decomposition import matrix_product_state
# tensor operations


#from math import ceil
#from google.colab import drive
#drive.mount('/content/gdrive')
import torch.nn.init as init
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader
import time
import os
import copy
import torch.nn.functional as Fun
print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
print("GPU is available?", torch.cuda.is_available())

PyTorch Version: 1.13.1
Torchvision Version: 0.14.1
GPU is available? True


In [2]:
dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
features = list()
with open("/home/c/cl237/TenBCD/UCI HAR/data/features.txt") as f:
    features = [line.split()[1] for line in f.readlines()]
print('# of Features: {}'.format(len(features)))

# of Features: 561


## get train data

In [4]:
pd_X_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_train.txt', delim_whitespace=True, header=None)
pd_X_train.columns = features
pd_y_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_train.txt', delim_whitespace=True, names=['Activity'])

N = len(pd_X_train)

K = 6

x = pd_X_train.values
y = pd_y_train.values

X_train=torch.tensor(x,dtype = torch.float32,device=device)
y_train=torch.tensor(y,dtype = int) #dtype=torch.int,device=device)

y_train = torch.flatten(y_train)

X_train = torch.t(X_train)

y_one_hot = torch.zeros(N, K).scatter_(1, torch.reshape(y_train-1, (N, 1)), 1)
y_one_hot = torch.t(y_one_hot).to(device=device)
y_train = y_train.to(device=device)

## get test data

In [5]:
pd_X_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_test.txt', delim_whitespace=True, header=None)
pd_X_test.columns = features
pd_y_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_test.txt', delim_whitespace=True, names=['Activity'])

N_test = len(pd_X_test)

x = pd_X_test.values
y = pd_y_test.values

X_test=torch.tensor(x,dtype = torch.float32, device=device)
y_test=torch.tensor(y,dtype = int)

y_test = torch.flatten(y_test)

X_test = torch.t(X_test)

y_test_one_hot = torch.zeros(N_test, K).scatter_(1, torch.reshape(y_test-1, (N_test, 1)), 1)
y_test_one_hot = torch.t(y_test_one_hot).to(device=device)
y_test = y_test.to(device=device)

In [6]:
def updateV(U1,U2,W,b,rho,gamma): 
    _, d = W.size()
    I = torch.eye(d, device=device) #Returns a 2-D tensor with ones on the diagonal and zeros elsewhere.
    U1 = nn.ReLU()(U1)
    _, col_U2 = U2.size()
    Vstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(W),W))+gamma*I), rho*torch.mm(torch.t(W),U2-b.repeat(1,col_U2))+gamma*U1)
    # check how to get V* W*
    # torch.mm*=(input,mat2) performs a matrix multiplication of the matrices input and mat2.
    #   if input is a (n*m) tensor,mat2 is a (m*p) tensor, output will be (n*p) tensor
    return Vstar

def updateWb_org(U, V, W, b, alpha, rho): 
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse(alpha*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

def updateWb(U, V, W, b, W_tensor_rec, alpha, rho,tau): 
    W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
    W_tensor2matrix = W_tensor_rec.reshape(W.shape)
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+tau*W_tensor2matrix+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse((alpha+tau)*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

# U update  E(1) in global convergent
def relu_prox(a, b, gamma, d, N):
    val = torch.empty(d,N, device=device)
    x = (a+gamma*b)/(1+gamma)
    y = torch.min(b,torch.zeros(d,N, device=device))

    val = torch.where(a+gamma*b < 0, y, torch.zeros(d,N, device=device))
    val = torch.where(((a+gamma*b >= 0) & (b >=0)) | ((a*(gamma-np.sqrt(gamma*(gamma+1))) <= gamma*b) & (b < 0)), x, val)
    val = torch.where((-a <= gamma*b) & (gamma*b <= a*(gamma-np.sqrt(gamma*(gamma+1)))), b, val)
    return val

In [9]:
########### parameter setup
#df = pd.DataFrame()
#df.to_csv('C:/Users/Mark/Desktop/HAR_5layers_rank40_100times.csv')
niter = 500
rank = 80
tau = 3
gamma = 0.5
rho = 0.5
alpha = 1



print ("rank=",rank, "tau=",tau, "gamma=",gamma, "rho=",rho, "alpha",alpha)

loss1 = np.empty(niter)
loss2 = np.empty(niter)
accuracy_train = np.empty(niter)
accuracy_test = np.empty(niter)
time1 = np.empty(niter)

results = torch.zeros(1, 5, niter)



for Out_iter in range(1):
    rank_initial = 400
    seed = 10 + 10*Out_iter
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)
    d0 = 561 #561 =3*11*17

    d1 =  1024    # 2^10
    d2 =  1024  
    d3 = 1024
    d4 = 512      # 2^9
    d5 = 512
    d6 = 6 


    W1 = 0.18*init.kaiming_uniform_(torch.empty(d1, d0, device=device), a=0, mode='fan_in', nonlinearity='leaky_relu')
    W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
    W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
    factors1 = tensor_train(W1_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W1_tl_tensor_rec = tt_to_tensor(factors1)
    b1 = 0*torch.ones(d1, 1, device=device) # 0 is stable


    W2 = 0.18*init.kaiming_uniform_(torch.empty(d2, d1, device=device), a=0, mode='fan_in', nonlinearity='leaky_relu')
    W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy())
    factors2 = tensor_train(W2_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    # we have 9 rank_initial (r1,...,r9) here plus 2 of 1s (r0 & r10).
    # factors: set of tensor cores
    # http://tensorly.org/stable/user_guide/tensor_decomposition.html
    # http://tensorly.org/stable/modules/generated/tensorly.decomposition.tensor_train.html#tensorly.decomposition.tensor_train
    W2_tl_tensor_rec = tt_to_tensor(factors2)
    # tt_to_tensor(factors):Re-assembles ‘factors’, which represent a tensor in TT format into the corresponding full tensor
    #      facros: list of 3d-arrays tt-cores           output_tensor: ndarray      
    b2 = 0*torch.ones(d2, 1, device=device)


    W3 = 0.18*init.kaiming_uniform_(torch.empty(d3, d2, device=device), a=0, mode='fan_in', nonlinearity='leaky_relu')
    W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
    factors3 = tensor_train(W3_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W3_tl_tensor_rec = tt_to_tensor(factors3)
    b3 = 0*torch.ones(d3, 1, device=device)

    W4 = 0.18*init.kaiming_uniform_(torch.empty(d4, d3, device=device), a=0, mode='fan_in', nonlinearity='leaky_relu')
    W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2)) # 8 number of 4s, 2 number of 8s
    W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    factors4 = tensor_train(W4_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W4_tl_tensor_rec = tt_to_tensor(factors4)
    b4 = 0*torch.ones(d4, 1, device=device)

    W5 = 0.18*init.kaiming_uniform_(torch.empty(d5, d4, device=device), a=0, mode='fan_in', nonlinearity='leaky_relu')
    W5_torch_tensor = W5.reshape((4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W5_tl_tensor = tl.tensor(W5_torch_tensor.cpu().numpy())
    factors5 = tensor_train(W5_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W5_tl_tensor_rec = tt_to_tensor(factors5)
    b5 = 0*torch.ones(d5, 1, device=device)


    W6 = 0.18*init.kaiming_uniform_(torch.empty(d6, d5, device=device), a=0, mode='fan_in', nonlinearity='leaky_relu')
    b6 = 0*torch.ones(d6, 1, device=device)

    # W4 = 0.01*torch.randn(d4, d3, device=device)
    # W4_torch_tensor = W4.reshape((50,30,K))
    # W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    # core4, tucker_factors4 = tucker(W4_tl_tensor, rank=(40,28,K), init='svd', tol=10e-5, random_state=12345)
    # W4_tl_tensor_rec = tl.tucker_to_tensor((core4, tucker_factors4))
    # b4 = 0.1*torch.ones(d4, 1, device=device)

    U1 = torch.addmm(b1.repeat(1, N), W1, X_train)
    V1 = nn.ReLU()(U1)
    U2 = torch.addmm(b2.repeat(1, N), W2, V1)
    V2 = nn.ReLU()(U2)
    U3 = torch.addmm(b3.repeat(1, N), W3, V2)
    V3 = nn.ReLU()(U3)
    U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    V4 = nn.ReLU()(U4)
    U5 = torch.addmm(b5.repeat(1, N), W5, V4)
    V5 = nn.ReLU()(U5)
    U6 = torch.addmm(b6.repeat(1, N), W6, V5)
    V6 = U6 
    # U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    # V4 = U4



    # Iterations
    print('Train on', N, 'samples, validate on', N_test, 'samples')
    for k in range(niter):
        start = time.time()

 # update for last layer
        # update V4
        V6 = (y_one_hot + gamma*U6 + alpha*V6)/(1 + gamma + alpha)

        # update U4 
        U6 = (gamma*V6 + rho*(torch.mm(W6,V5) + b6.repeat(1,N)))/(gamma + rho)

        # update W4 and b4
        W6, b6 = updateWb_org(U6,V5,W6,b6,alpha,rho)
        # W3_torch_tensor = W3.reshape((d3,4,4,4,4,4,4))
        # W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
        # factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,1))
        # W3_tl_tensor_rec = tt_to_tensor(factors3)


        # update for 5th layer
        # update V3
        V5 = updateV(U5,U6,W6,b6,rho,gamma)

        # update U3
        U5 = relu_prox(V5,(rho*torch.addmm(b5.repeat(1,N), W5, V4) + alpha*U5)/(rho + alpha),(rho + alpha)/gamma,d5,N)

        # update W3 and b3
        W5, b5 = updateWb(U5,V4,W5,b5,W5_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W5_torch_tensor = W5.reshape((4,4,4,4,4,4,4,4,4))
        W5_tl_tensor = tl.tensor(W5_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors5 = tensor_train(W5_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W5_tl_tensor_rec = tt_to_tensor(factors5)


  # update for 4th layer
        # update V3
        V4 = updateV(U4,U5,W5,b5,rho,gamma)

        # update U3
        U4 = relu_prox(V4,(rho*torch.addmm(b4.repeat(1,N), W4, V3) + alpha*U4)/(rho + alpha),(rho + alpha)/gamma,d4,N)

        # update W3 and b3
        W4, b4 = updateWb(U4,V3,W4,b4,W4_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2))
        W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors4 = tensor_train(W4_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W4_tl_tensor_rec = tt_to_tensor(factors4)

  # update for 3nd layer
        # update V3
        V3 = updateV(U3,U4,W4,b4,rho,gamma)

        # update U3
        U3 = relu_prox(V3,(rho*torch.addmm(b3.repeat(1,N), W3, V2) + alpha*U3)/(rho + alpha),(rho + alpha)/gamma,d3,N)

        # update W3 and b3
        W3, b3 = updateWb(U3,V2,W3,b3,W3_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4))
        W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W3_tl_tensor_rec = tt_to_tensor(factors3)

 # update for 2nd layer
        # update V2
        V2 = updateV(U2,U3,W3,b3,rho,gamma)

        # update U2
        U2 = relu_prox(V2,(rho*torch.addmm(b2.repeat(1,N), W2, V1) + alpha*U2)/(rho + alpha),(rho + alpha)/gamma,d2,N)

        # update W2 and b2
        W2, b2 = updateWb(U2,V1,W2,b2,W2_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4))
        W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy()) 
        factors2 = tensor_train(W2_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W2_tl_tensor_rec = tt_to_tensor(factors2)


# update for 1st layer
        # update V1
        V1 = updateV(U1,U2,W2,b2,rho,gamma)

        # update U1
        U1 = relu_prox(V1,(rho*torch.addmm(b1.repeat(1,N), W1, X_train) + alpha*U1)/(rho + alpha),(rho + alpha)/gamma,d1,N)

        # update W1 and b1
        W1, b1 = updateWb(U1,X_train,W1,b1,W1_tl_tensor_rec, alpha,rho,tau)

        # G update
        W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
        W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
        factors1 = tensor_train(W1_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W1_tl_tensor_rec = tt_to_tensor(factors1)




 # prediction for trainning data
        a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_train))
        #a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train))
        a4_train = nn.ReLU()(torch.addmm(b4.repeat(1, N), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_train))
        a5_train = nn.ReLU()(torch.addmm(b5.repeat(1, N), torch.as_tensor(W5_tl_tensor_rec,device=device).reshape((d5, d4)).float(), a4_train))
        #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        pred = torch.argmax(torch.addmm(b6.repeat(1, N), W6, a5_train), dim=0)
        # check argmax and addmm, dim=0

 #Prediction for test data
        a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_test))
        #a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test))
        a4_test = nn.ReLU()(torch.addmm(b4.repeat(1, N_test), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_test))
        a5_test = nn.ReLU()(torch.addmm(b5.repeat(1, N_test), torch.as_tensor(W5_tl_tensor_rec,device=device).reshape((d5, d4)).float(), a4_test))
        pred_test = torch.argmax(torch.addmm(b6.repeat(1, N_test), W6, a5_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        # # a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_train))
        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        # # a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), W2, a1_train))

        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), W3, a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), W2, a1_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), W3, a2_test), dim=0)




   #emperical loss
        loss1[k] = gamma/2*torch.pow(torch.dist(V6,y_one_hot,2),2).cpu().numpy()
        # torch.pow:Takes the power of each element in input with exponent and returns a tensor with the result.
        # torch.disk: Returns the p-norm of (input - other)
        # Eq (5) in paper
        loss2[k] = loss1[k] + rho/2*torch.pow(torch.dist(torch.addmm(b1.repeat(1,N), W1, X_train),U1,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b2.repeat(1,N), W2, V1),U2,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b3.repeat(1,N), W3, V2),U3,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b4.repeat(1,N), W4, V3),U4,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b5.repeat(1,N), W5, V4),U5,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b6.repeat(1,N), W6, V5),U6,2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V1,nn.ReLU()(U1),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V2,nn.ReLU()(U2),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V3,nn.ReLU()(U3),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V4,nn.ReLU()(U4),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V5,nn.ReLU()(U5),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V6,U6,2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W1.reshape((6,22,34,2,2,2,2,2,2,2)),torch.as_tensor(W1_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W2.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W2_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W3.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W4.reshape((4,4,4,4,4,4,4,4,4,2)),torch.as_tensor(W4_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W5.reshape((4,4,4,4,4,4,4,4,4)),torch.as_tensor(W5_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \

        # +tau/2*torch.pow(torch.dist(W3.reshape((d3,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() 
        # +tau/2*torch.pow(torch.dist(W4.reshape((50,30,K)),torch.as_tensor(W4_tl_tensor_rec,device=device),2),2).cpu().numpy() 

        # compute training accuracy
        correct_train = pred == y_train-1
        accuracy_train[k] = np.mean(correct_train.cpu().numpy())

        # compute validation accuracy
        correct_test = pred_test == y_test-1
        accuracy_test[k] = np.mean(correct_test.cpu().numpy())

        # compute training time
        stop = time.time()
        duration = stop - start
        time1[k] = duration

        # print results
        print('Repeatition', Out_iter + 1, 'Epoch', k + 1, '/', niter, '\n', 
              '-', 'time:', time1[k], '-', 'sq_loss:', loss1[k], '-', 'tot_loss:', loss2[k], 
              '-', 'acc:', accuracy_train[k], '-', 'val_acc:', accuracy_test[k])

##############
############## compute CR
    factors1_shape=[f.shape for f in factors1]
    Sum_of_variables_factors1=sum(list(x*y*z for x,y,z in factors1_shape))
    factors2_shape=[f.shape for f in factors2]
    Sum_of_variables_factors2=sum(list(x*y*z for x,y,z in factors2_shape))
    factors3_shape=[f.shape for f in factors3]
    Sum_of_variables_factors3=sum(list(x*y*z for x,y,z in factors3_shape))
    factors4_shape=[f.shape for f in factors4]
    Sum_of_variables_factors4=sum(list(x*y*z for x,y,z in factors4_shape))
    factors5_shape=[f.shape for f in factors5]
    Sum_of_variables_factors5=sum(list(x*y*z for x,y,z in factors5_shape))

    total_variabels=Sum_of_variables_factors1+Sum_of_variables_factors2+Sum_of_variables_factors3+Sum_of_variables_factors4+Sum_of_variables_factors5

    CR_1=((total_variabels)+(d5*d6))/(d0*d1+d1*d2+d2*d3+d3*d4+d4*d5+d5*d6)
    CR_2=(total_variabels)/(d0*d1+d1*d2+d2*d3+d3*d4+d4*d5)
    print("CR_1 = "+str(CR_1)+"   "+"CR_2 = " +str(CR_2))


    results[Out_iter,0,:] = torch.tensor(loss1)
    results[Out_iter,1,:] = torch.tensor(loss2)
    results[Out_iter,2,:] = torch.tensor(accuracy_train)
    results[Out_iter,3,:] = torch.tensor(accuracy_test)
    results[Out_iter,4,:] = torch.tensor(time1)
    CR=(CR_1,CR_2)

#     #this postion to add new row into existing table
#         df=pd.read_csv('C:/Users/Mark/Desktop/HAR_5layers_rank40_100times.csv')
#         new_row = {'rank':rank, 'CR_1':CR_1, 'CR_2':CR_2, 'tau':tau, 'gamma':gamma, 'rho':rho, 'alpha':alpha, \
#                    'loss1':loss1[niter-1], 'loss2':loss2[niter-1], 'accuracy_train':accuracy_train[niter-1], \
#                    'accuracy_test':accuracy_test[niter-1], 'time':time1[niter-1],'seed':seed} 
#         df=df.append(new_row,ignore_index=True)
#         df.to_csv('C:/Users/Mark/Desktop/HAR_5layers_rank40_100times.csv',index=False)


filename= "KaimingUniform_" + "niter_"+ str(niter) + "rank_" + str(rank) + "tau_" + str(tau) + "gamma_" + str(gamma) + \
"rho_" + str(rho) + "alpha_" + str(alpha) + ".mat"
from scipy.io import savemat
%cd /home/c/cl237/TenBCD/UCI HAR/5 hidden layers/Different Initialization/
savemat (filename, {'results': torch.Tensor.numpy(results), 'Compression Ratio':CR})
#this position to save table into matlab
#df.to_csv('C:/Users/Mark/Desktop/result_compress_123_layer_highrank.csv')
##Changing Folder
 #%cd '/content/gdrive/MyDrive/Colab/Tensor-BCD-for-DNN/Experiments/DataSaved'
 #savemat("ThreeLayer_rank160.mat", {'results': torch.Tensor.numpy(results)})    

rank= 80 tau= 3 gamma= 0.5 rho= 0.5 alpha 1
Train on 7352 samples, validate on 2947 samples
Repeatition 1 Epoch 1 / 500 
 - time: 1.578333854675293 - sq_loss: 661.652099609375 - tot_loss: 1062.3111628236802 - acc: 0.22755712731229596 - val_acc: 0.20597217509331522
Repeatition 1 Epoch 2 / 500 
 - time: 1.5063374042510986 - sq_loss: 294.0675354003906 - tot_loss: 507.8709136461839 - acc: 0.27189880304679 - val_acc: 0.2653545978961656
Repeatition 1 Epoch 3 / 500 
 - time: 1.519254207611084 - sq_loss: 162.49237060546875 - tot_loss: 280.48820755816996 - acc: 0.22986942328618062 - val_acc: 0.21954530030539532
Repeatition 1 Epoch 4 / 500 
 - time: 1.5444300174713135 - sq_loss: 88.346923828125 - tot_loss: 161.71768063586205 - acc: 0.1894722524483134 - val_acc: 0.18018323719036308
Repeatition 1 Epoch 5 / 500 
 - time: 1.5257132053375244 - sq_loss: 47.59058380126953 - tot_loss: 97.3372645676136 - acc: 0.1913764961915125 - val_acc: 0.18221920597217509
Repeatition 1 Epoch 6 / 500 
 - time: 1.561008

Repeatition 1 Epoch 48 / 500 
 - time: 1.5355300903320312 - sq_loss: 0.0003618234768509865 - tot_loss: 2.1230737952209893 - acc: 0.764689880304679 - val_acc: 0.7254835425856804
Repeatition 1 Epoch 49 / 500 
 - time: 1.5146212577819824 - sq_loss: 0.0003373716026544571 - tot_loss: 2.080036603863846 - acc: 0.7676822633297062 - val_acc: 0.7295554801493044
Repeatition 1 Epoch 50 / 500 
 - time: 1.5463201999664307 - sq_loss: 0.0003147937823086977 - tot_loss: 1.985768975224346 - acc: 0.7708106637649619 - val_acc: 0.7332880895826264
Repeatition 1 Epoch 51 / 500 
 - time: 1.5298166275024414 - sq_loss: 0.0002940105623565614 - tot_loss: 1.924092289773398 - acc: 0.7742110990206746 - val_acc: 0.7343060739735324
Repeatition 1 Epoch 52 / 500 
 - time: 1.5369107723236084 - sq_loss: 0.00027486670296639204 - tot_loss: 1.858865196334591 - acc: 0.7766594124047879 - val_acc: 0.7380386834068544
Repeatition 1 Epoch 53 / 500 
 - time: 1.540961742401123 - sq_loss: 0.00025712253409437835 - tot_loss: 1.814722807

Repeatition 1 Epoch 95 / 500 
 - time: 1.5365240573883057 - sq_loss: 2.2852857000543736e-05 - tot_loss: 0.6758381530571569 - acc: 0.8792165397170838 - val_acc: 0.8201560909399389
Repeatition 1 Epoch 96 / 500 
 - time: 1.5289278030395508 - sq_loss: 2.1750716769020073e-05 - tot_loss: 0.6470453441211248 - acc: 0.8804406964091404 - val_acc: 0.821174075330845
Repeatition 1 Epoch 97 / 500 
 - time: 1.50816011428833 - sq_loss: 2.0680938177974895e-05 - tot_loss: 0.6499386846801372 - acc: 0.8819368879216539 - val_acc: 0.823549372242959
Repeatition 1 Epoch 98 / 500 
 - time: 1.5091423988342285 - sq_loss: 1.9696028175530955e-05 - tot_loss: 0.6485999948811241 - acc: 0.8831610446137106 - val_acc: 0.8245673566338649
Repeatition 1 Epoch 99 / 500 
 - time: 1.5410206317901611 - sq_loss: 1.877481372503098e-05 - tot_loss: 0.6157357398565182 - acc: 0.8847932535364527 - val_acc: 0.825246012894469
Repeatition 1 Epoch 100 / 500 
 - time: 1.562300205230713 - sq_loss: 1.7889882656163536e-05 - tot_loss: 0.61987

Repeatition 1 Epoch 141 / 500 
 - time: 1.5575246810913086 - sq_loss: 4.149294454691699e-06 - tot_loss: 0.3900971885162505 - acc: 0.9183895538628944 - val_acc: 0.8703766542246352
Repeatition 1 Epoch 142 / 500 
 - time: 1.5306744575500488 - sq_loss: 4.062597326992545e-06 - tot_loss: 0.3949882592582412 - acc: 0.919069640914037 - val_acc: 0.8717339667458432
Repeatition 1 Epoch 143 / 500 
 - time: 1.535310983657837 - sq_loss: 3.981378995376872e-06 - tot_loss: 0.358503751802175 - acc: 0.9194776931447225 - val_acc: 0.8720732948761453
Repeatition 1 Epoch 144 / 500 
 - time: 1.5357539653778076 - sq_loss: 3.899241619365057e-06 - tot_loss: 0.36373997633344857 - acc: 0.9200217627856365 - val_acc: 0.8724126230064473
Repeatition 1 Epoch 145 / 500 
 - time: 1.5430245399475098 - sq_loss: 3.816328899119981e-06 - tot_loss: 0.38949571572906905 - acc: 0.9207018498367792 - val_acc: 0.8727519511367492
Repeatition 1 Epoch 146 / 500 
 - time: 1.5340983867645264 - sq_loss: 3.744715058928705e-06 - tot_loss: 0.

Repeatition 1 Epoch 187 / 500 
 - time: 1.5631699562072754 - sq_loss: 2.375923486397369e-06 - tot_loss: 0.283376238364653 - acc: 0.9389281828073993 - val_acc: 0.9012555140821175
Repeatition 1 Epoch 188 / 500 
 - time: 1.5498361587524414 - sq_loss: 2.3586421775689814e-06 - tot_loss: 0.3015429371363467 - acc: 0.9396082698585418 - val_acc: 0.9032914828639295
Repeatition 1 Epoch 189 / 500 
 - time: 1.5698273181915283 - sq_loss: 2.3418024284183048e-06 - tot_loss: 0.32238074361115565 - acc: 0.9397442872687704 - val_acc: 0.9039701391245334
Repeatition 1 Epoch 190 / 500 
 - time: 1.567765712738037 - sq_loss: 2.3268962650035974e-06 - tot_loss: 0.3111160112984592 - acc: 0.9398803046789989 - val_acc: 0.9039701391245334
Repeatition 1 Epoch 191 / 500 
 - time: 1.5888638496398926 - sq_loss: 2.31744843404158e-06 - tot_loss: 0.30473634475181655 - acc: 0.9405603917301415 - val_acc: 0.9039701391245334
Repeatition 1 Epoch 192 / 500 
 - time: 1.5811541080474854 - sq_loss: 2.3072984731697943e-06 - tot_loss

...

Repeatition 1 Epoch 279 / 500 
 - time: 1.5100905895233154 - sq_loss: 1.695276182545058e-06 - tot_loss: 0.2435681880511078 - acc: 0.9640914036996736 - val_acc: 0.9256871394638616
Repeatition 1 Epoch 280 / 500 
 - time: 1.5295839309692383 - sq_loss: 1.6917774701141752e-06 - tot_loss: 0.23541617812637838 - acc: 0.9646354733405876 - val_acc: 0.9256871394638616
Repeatition 1 Epoch 281 / 500 
 - time: 1.5235512256622314 - sq_loss: 1.6881493820619653e-06 - tot_loss: 0.24447600694172245 - acc: 0.9646354733405876 - val_acc: 0.9256871394638616
Repeatition 1 Epoch 282 / 500 
 - time: 1.5493237972259521 - sq_loss: 1.6859498828125652e-06 - tot_loss: 0.2612692066479356 - acc: 0.9647714907508161 - val_acc: 0.9267051238547676
Repeatition 1 Epoch 283 / 500 
 - time: 1.5523765087127686 - sq_loss: 1.6807440488264547e-06 - tot_loss: 0.24354366918990245 - acc: 0.9647714907508161 - val_acc: 0.9270444519850696
Repeatition 1 Epoch 284 / 500 
 - time: 1.5190036296844482 - sq_loss: 1.6751105249568354e-06 - tot

Repeatition 1 Epoch 325 / 500 
 - time: 1.6026277542114258 - sq_loss: 1.5197263110167114e-06 - tot_loss: 0.2646959993636333 - acc: 0.971164309031556 - val_acc: 0.9321343739395996
Repeatition 1 Epoch 326 / 500 
 - time: 1.5627024173736572 - sq_loss: 1.517013629381836e-06 - tot_loss: 0.264669472047232 - acc: 0.971436343852013 - val_acc: 0.9324737020699015
Repeatition 1 Epoch 327 / 500 
 - time: 1.552095651626587 - sq_loss: 1.5145415090955794e-06 - tot_loss: 0.24930346074830645 - acc: 0.9715723612622416 - val_acc: 0.9324737020699015
Repeatition 1 Epoch 328 / 500 
 - time: 1.5600299835205078 - sq_loss: 1.5096683227966423e-06 - tot_loss: 0.28992743313868274 - acc: 0.9718443960826986 - val_acc: 0.9324737020699015
Repeatition 1 Epoch 329 / 500 
 - time: 1.5246543884277344 - sq_loss: 1.5056735946927802e-06 - tot_loss: 0.2532202761570339 - acc: 0.9717083786724701 - val_acc: 0.9328130302002036
Repeatition 1 Epoch 330 / 500 
 - time: 1.5635347366333008 - sq_loss: 1.5030158238005242e-06 - tot_loss

Repeatition 1 Epoch 371 / 500 
 - time: 1.5043187141418457 - sq_loss: 1.3816587625115062e-06 - tot_loss: 0.24747989825701833 - acc: 0.9749727965179543 - val_acc: 0.9348489989820156
Repeatition 1 Epoch 372 / 500 
 - time: 1.5452423095703125 - sq_loss: 1.3791614037472755e-06 - tot_loss: 0.25739010538980844 - acc: 0.9751088139281828 - val_acc: 0.9351883271123176
Repeatition 1 Epoch 373 / 500 
 - time: 1.4992966651916504 - sq_loss: 1.3760086403635796e-06 - tot_loss: 0.28896592722042813 - acc: 0.9751088139281828 - val_acc: 0.9348489989820156
Repeatition 1 Epoch 374 / 500 
 - time: 1.5565192699432373 - sq_loss: 1.37273980271857e-06 - tot_loss: 0.23576797247196613 - acc: 0.9752448313384113 - val_acc: 0.9348489989820156
Repeatition 1 Epoch 375 / 500 
 - time: 1.526789665222168 - sq_loss: 1.3695730558538344e-06 - tot_loss: 0.2443896600538764 - acc: 0.9752448313384113 - val_acc: 0.9355276552426196
Repeatition 1 Epoch 376 / 500 
 - time: 1.5285792350769043 - sq_loss: 1.3660511513080564e-06 - tot_

Repeatition 1 Epoch 417 / 500 
 - time: 1.525364637374878 - sq_loss: 1.2741688806272577e-06 - tot_loss: 0.27433306179865724 - acc: 0.9785092491838956 - val_acc: 0.9395995928062436
Repeatition 1 Epoch 418 / 500 
 - time: 1.5605030059814453 - sq_loss: 1.2710212331512594e-06 - tot_loss: 0.2788358410156193 - acc: 0.9785092491838956 - val_acc: 0.9389209365456397
Repeatition 1 Epoch 419 / 500 
 - time: 1.5466809272766113 - sq_loss: 1.2693615190073615e-06 - tot_loss: 0.2378488039329718 - acc: 0.9786452665941241 - val_acc: 0.9385816084153377
Repeatition 1 Epoch 420 / 500 
 - time: 1.5655007362365723 - sq_loss: 1.2677738823185791e-06 - tot_loss: 0.24204416038473386 - acc: 0.9787812840043526 - val_acc: 0.9379029521547336
Repeatition 1 Epoch 421 / 500 
 - time: 1.564507246017456 - sq_loss: 1.264866682504362e-06 - tot_loss: 0.2639622797305887 - acc: 0.9785092491838956 - val_acc: 0.9385816084153377
Repeatition 1 Epoch 422 / 500 
 - time: 1.545454978942871 - sq_loss: 1.2613712669917732e-06 - tot_los

Repeatition 1 Epoch 463 / 500 
 - time: 1.5394253730773926 - sq_loss: 1.1757907714127214e-06 - tot_loss: 0.2856916470368085 - acc: 0.9806855277475517 - val_acc: 0.9426535459789617
Repeatition 1 Epoch 464 / 500 
 - time: 1.562849521636963 - sq_loss: 1.174335011455696e-06 - tot_loss: 0.2296665353148839 - acc: 0.9806855277475517 - val_acc: 0.9429928741092637
Repeatition 1 Epoch 465 / 500 
 - time: 1.5673339366912842 - sq_loss: 1.1731010545190657e-06 - tot_loss: 0.2748834876592321 - acc: 0.9805495103373232 - val_acc: 0.9426535459789617
Repeatition 1 Epoch 466 / 500 
 - time: 1.5761148929595947 - sq_loss: 1.1708992815329111e-06 - tot_loss: 0.2344555945600093 - acc: 0.9813656147986942 - val_acc: 0.9433322022395657
Repeatition 1 Epoch 467 / 500 
 - time: 1.5525107383728027 - sq_loss: 1.1697030686264043e-06 - tot_loss: 0.2753794760101025 - acc: 0.9806855277475517 - val_acc: 0.9429928741092637
Repeatition 1 Epoch 468 / 500 
 - time: 1.5395011901855469 - sq_loss: 1.1689596703945426e-06 - tot_los

In [8]:
seed

10